In [11]:
import os 
import numpy  as np
import cv2
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D

In [3]:
parasitized =r"C:\Users\DELL\Desktop\DEEP_Learning\cell_images\Parasitized"
Uninfected  = r"C:\Users\DELL\Desktop\DEEP_Learning\cell_images\Uninfected"

In [4]:
X = []
y = []

In [6]:
for image in os.listdir(parasitized):
    try:
        img = cv2.imread(rf"{parasitized}\{image}")
        img = cv2.resize(img,(60,60))
        img = img/255
        X.append(img)
        y.append(1)
        
    except:
        pass
    

In [7]:
print(len(X))
print(len(y))

13104
13104


In [8]:
for image in os.listdir(Uninfected):
    try:
        img = cv2.imread(rf"{Uninfected}\{image}")
        img = cv2.resize(img,(60,60))
        img = img/255
        X.append(img)
        y.append(0)
        
    except:
        pass

In [9]:
print(len(X))
print(len(y))

26186
26186


In [12]:
X_arr = np.array(X)
y_arr = np.array(y)

In [13]:
from sklearn.model_selection import train_test_split 


In [15]:
X_train,X_test,y_train,y_test = train_test_split(X_arr,y_arr,test_size=0.25,random_state=1)

In [16]:
from keras.applications.mobilenet_v2 import MobileNetV2

In [17]:
model = MobileNetV2(input_shape=(60,60,3),include_top=False)
model.summary()

9406464/9406464 [==============================] - 83s 9us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 60, 60, 3)]          0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 30, 30, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 30, 30, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (Re

In [18]:
mod = Sequential()
mod.add(model)
mod.add(Flatten())
mod.add(Dense(64,activation= 'relu'))
mod.add(Dense(128,activation= 'relu'))
mod.add(Dense(1,activation= 'sigmoid'))

mod.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 2, 2, 1280)        2257984   
 tional)                                                         
                                                                 
 flatten (Flatten)           (None, 5120)              0         
                                                                 
 dense (Dense)               (None, 64)                327744    
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2594177 (9.90 MB)
Trainable params: 2560065 (9.77 MB)
Non-trainable params: 34112 (133.25 KB)
______________

In [19]:
mod.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])



In [21]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss',min_delta=0.002,patience= 10,verbose=1,restore_best_weights=True,mode='min')

In [22]:
mod.fit(X_train,y_train,validation_split=0.2,epochs=50,callbacks=[es])

Epoch 1/50
491/491 [==============================] - 232s 427ms/step - loss: 0.1921 - accuracy: 0.9375 - val_loss: 0.6985 - val_accuracy: 0.9127
Epoch 2/50
491/491 [==============================] - 190s 387ms/step - loss: 0.1214 - accuracy: 0.9586 - val_loss: 4.9590 - val_accuracy: 0.4885
Epoch 3/50
491/491 [==============================] - 189s 385ms/step - loss: 0.1098 - accuracy: 0.9637 - val_loss: 1.7094 - val_accuracy: 0.4689
Epoch 4/50
491/491 [==============================] - 243s 494ms/step - loss: 0.1145 - accuracy: 0.9602 - val_loss: 0.6831 - val_accuracy: 0.9173
Epoch 5/50
491/491 [==============================] - 249s 507ms/step - loss: 0.1223 - accuracy: 0.9584 - val_loss: 27.8174 - val_accuracy: 0.5056
Epoch 6/50
491/491 [==============================] - 249s 507ms/step - loss: 0.1178 - accuracy: 0.9617 - val_loss: 0.5946 - val_accuracy: 0.8809
Epoch 7/50
491/491 [==============================] - 245s 498ms/step - loss: 0.1071 - accuracy: 0.9619 - val_loss: 5.1436 

In [23]:
mod.evaluate(X_test,y_test)


205/205 [==============================] - 23s 78ms/step - loss: 0.3973 - accuracy: 0.9502


[0.39725348353385925, 0.9502062201499939]

In [24]:
img = cv2.imread(r'C:\Users\DELL\Desktop\DEEP_Learning\cell_images\Parasitized\C33P1thinF_IMG_20150619_114756a_cell_180.png')
img = cv2.resize(img,(60,60))
img =  img/255
img = img.reshape((1,)+img.shape)
img.shape


(1, 60, 60, 3)

In [25]:
int(mod.predict(img)>=0.5)

1/1 [==============================] - 12s 12s/step


1